In [300]:
import numpy as np
import pandas as pd
import json
import requests
import operator
from google.cloud import translate

In [279]:
country_code = 0
req = requests.get("https://us-central1-vision-migration.cloudfunctions.net/la_hacks_2019?market_code="+str(country_code))
json1 = json.loads(req.content)

In [141]:
buckets = 3
traffic_names = {}
for i in range(buckets):
    report = json1['buckets'][i]['report']
    rollups = report['rollups']
    for j in range(len(rollups)):
        name = rollups[j]['name'] 
        similarity = rollups[j]['similarity']
        if name not in traffic_names:
            traffic_names[name] = rollups[j]['traffic']['totalTraffic']
        else:
            traffic_names[name] += rollups[j]['traffic']['totalTraffic']
        
sorted_traffic_names = sorted(traffic_names.items(),key=operator.itemgetter(1),reverse=True)   

In [163]:
name_mapping = {}
for bucket in range(buckets):
    report = json1['buckets'][bucket]['report']
    rollups = report['rollups']
    for topic in rollups:
        name = topic['name']
        similarity = topic['similarity']
        urls_temp = topic['top_articles_on_network']
        urls = []
        for temp in urls_temp:
            urls = list(set(urls))+list(temp.keys())
        if name not in name_mapping:
            name_mapping[name] = {'urls':urls,'similarity':{}}
        else:
            for key in similarity:
                if key not in name_mapping[name]['similarity']:
                    name_mapping[name]['similarity'][key] = [similarity[key]]
                else:
                    name_mapping[name]['similarity'][key].append(similarity[key])
            
for key in name_mapping:
    similarity_list = name_mapping[key]['similarity']
    for neighbour in similarity_list:
        similarity_list[neighbour] = np.mean(similarity_list[neighbour])
        
threshold = 0.2
neighbor_set = set()
final_top = []

for name, no in sorted_traffic_names:
    if name not in neighbor_set:
        dictx = name_mapping[name]
        final_top.append({name:dictx})
        
        for key, val in dictx['similarity'].items():
            if val >= threshold:
                neighbor_set.add(key)

In [3]:
'''# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Brother Of Suspected Killer Trolls Trump: Obstruction Charges Can Make World "Safer"'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))'''

Text: Trump Rape cuts all direct assistance to Honduras, El Salvador, Guatemala
Sentiment: -0.5, 0.5


In [185]:
def translateText(text, target):
    translate_client = translate.Client()
    translation = translate_client.translate(text,target_language=target)
    return translation['translatedText']

In [196]:
# The text to translate
text = u'In a stunning about-face, State Department officials said that President Donald Trump is cutting off all \
        direct assistance to the so-called Northern Triangle countries of El Salvador, Honduras and Guatemala.\n“At \
        the Secretary’s instruction, we are carrying out the President’s direction and ending FY [fiscal year] 2017 and \
        FY 2018 foreign assistance programs for the Northern Triangle,” a State Department spokesperson told ABC News, \
        referring to U.S. Secretary of State Mike Pompeo.\ntackle the problems facing our shared neighborhood,” Pence \
        said in October when hosting leaders from the Northern Triangle and Mexico at the State Department with Pompeo.\n'
target = 'hi'
start = time.time()
translated_text = translateText(text,target)
end = time.time()

In [198]:
print("Finished Translating in %0.3f seconds\n" %(end-start))
print("Text in English is:\n")
print(text+"\n")
print("Text in %s Language is:\n" %target)
print(translated_text)

Finished Translating in 1.769 seconds

Text in English is:

In a stunning about-face, State Department officials said that President Donald Trump is cutting off all         direct assistance to the so-called Northern Triangle countries of El Salvador, Honduras and Guatemala.
“At         the Secretary’s instruction, we are carrying out the President’s direction and ending FY [fiscal year] 2017 and         FY 2018 foreign assistance programs for the Northern Triangle,” a State Department spokesperson told ABC News,         referring to U.S. Secretary of State Mike Pompeo.
tackle the problems facing our shared neighborhood,” Pence         said in October when hosting leaders from the Northern Triangle and Mexico at the State Department with Pompeo.


Text in hi Language is:

एक तेजस्वी चेहरे के बारे में, विदेश विभाग के अधिकारियों ने कहा कि राष्ट्रपति डोनाल्ड ट्रम्प अल सल्वाडोर, होंडुरास और ग्वाटेमाला के तथाकथित उत्तरी त्रिभुज देशों के लिए सभी प्रत्यक्ष सहायता काट रहे हैं। विदेश विभाग के प